In [10]:
import pandas as pd
import numpy as np
import re
import json

In [20]:
df = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/b270ef1a-c219-48b1-8399-b1458e225d39/download/14_iscrittixresidenzasedecorsogruppo.csv', delimiter=';')

decoder = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/eae4ee94-0797-41d2-b007-bc6dad3ef3e2/download/07_iscrittixresidenza.csv', delimiter=';')

with open('decoders.json','r') as file:
    decoders = json.load(file)

cod_prov = decoders[0]
prov_cod = decoders[1]

df = df[df['AnnoA']=='2021/2022']

df['ResidenzaCOD'] = df['ResidenzaR'].apply(lambda x: prov_cod[x])

In [21]:
with open('cod_confini.json','r') as file:
    dec_conf = json.load(file)

In [22]:
filtro = [] 
for i,value in df.iterrows():
    prov = value['ResidenzaCOD']
    confini = dec_conf.get(str(prov),[])
    is_confinante = value['SedeP'] in confini 
    filtro.append(is_confinante)

In [23]:
pendolari = df[filtro]

In [24]:
pendolari['sede_ateneo'] = pendolari['SedeP'].apply(lambda x: cod_prov[str(x)])

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df['sede_ateneo'] = df['SedeP'].apply(lambda x: cod_prov[str(x)])

In [26]:
fuorisede = df[df['SedeP']!=df['ResidenzaCOD']]

In [27]:
fuorisede['sede_ateneo']=fuorisede['SedeP'].apply(lambda x: cod_prov[str(x)])

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Uni con più pendolari

In [28]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_p = pendolari[pendolari['AteneoNOME']==uni]['Isc'].sum() #numero pendolari
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    r = n_p/n_t*100
    unis.append(uni)
    rapps.append(r)

In [29]:
uni_pendolari = pd.DataFrame({'uni':unis,
             'pendolari/totale':rapps}).sort_values(by='pendolari/totale', ascending=False)

In [15]:
uni_pendolari.to_csv('pendolari_per_uni.csv', index=False)

# Adesso cerchiamo province che attirano più pendolari!

In [21]:
provs = []
rapps = []
for prov in df['sede_ateneo'].unique():
    n_pendolari = pendolari[pendolari['sede_ateneo']==prov]['Isc'].sum()
    n_tot = df[df['sede_ateneo']==prov]['Isc'].sum()
    if n_tot > 1000:
        r = n_pendolari/n_tot*100
        provs.append(prov)
        rapps.append(r)

In [22]:
pendolari_tot = pd.DataFrame({
    'province':provs,
    'pendolari/totale':rapps
}).sort_values(by='pendolari/totale',ascending=False)

In [23]:
pendolari_tot['cod'] = pendolari_tot['province'].apply(lambda x: prov_cod[x])

In [24]:
pendolari_tot.shape

(101, 3)

In [140]:
pendolari_tot.to_csv('province_con_più_pendolari.csv',index=False)

# Uni con più fuorisede (NON REAL)

In [30]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_f = fuorisede[fuorisede['AteneoNOME']==uni]['Isc'].sum()
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    
    if n_f <0:
        raise Exception('Qualcosa non va')
    r = n_f/n_t*100
    unis.append(uni)
    rapps.append(r)

In [31]:
uni_fuorisede = pd.DataFrame({
    'Unis':unis,
    'fuorisede/totale':rapps,
}).sort_values(by='fuorisede/totale',ascending=False)

In [36]:
uni_fuorisede.to_csv('fuorisede_per_uni.csv',index=False)

# ora facciamo real fuorisede: cioè fuorisede generali - pendolari

# Uni con più real fuorisede

In [32]:
unis = []
rapps = []
for uni in df['AteneoNOME'].unique():
    n_p = pendolari[pendolari['AteneoNOME']==uni]['Isc'].sum() #numero pendolari
    n_f = fuorisede[fuorisede['AteneoNOME']==uni]['Isc'].sum()
    n_t = df[df['AteneoNOME']==uni]['Isc'].sum() #numero totali
    real_fuorisede = n_f - n_p
    
    if real_fuorisede <0:
        raise Exception('Qualcosa non va')
    r = real_fuorisede/n_t*100
    unis.append(uni)
    rapps.append(r)

In [33]:
uni_fuorisede_real = pd.DataFrame({
    'Unis':unis,
    'real_fuorisede/totale':rapps,
}).sort_values(by='real_fuorisede/totale',ascending=False)

In [22]:
uni_fuorisede

,Unis,real_fuorisede/totale
80,Bra Scienze Gastronomiche,87.704918
28,Milano Bocconi,84.336177
67,Venezia Iuav,83.107769
77,Siena Stranieri,81.858164
55,Ferrara,80.064565
...,...,...
14,Brescia,23.460671
12,Roma Tre,23.178526
56,Reggio Calabria,19.535284
7,Napoli Parthenope,17.294128


In [16]:
uni_fuorisede.to_csv('real_fuorisede_per_uni.csv',index=False)

# Province con più real fuorisede

In [100]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique(): #mettiamo come elenco unique sempre province di residenza 
                                        #perché più completo
    n_pendolari = pendolari[pendolari['sede_ateneo']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['sede_ateneo']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
        
    n_tot = df[df['sede_ateneo']==prov]['Isc'].sum()
    #if n_tot > 1000:
    r = real_fuorisede/n_tot*100
    provs.append(prov)
    rapps.append(r)

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  


In [101]:
arrivi_real = pd.DataFrame({
    'province':provs,
    'real_fuorisede/totale':rapps
}).sort_values(by='real_fuorisede/totale',ascending=False).fillna(0)

arrivi_real['cod'] = arrivi_real['province'].apply(lambda x: prov_cod[x])
arrivi_real.to_csv('province_accolgono_real_fuorisede.csv',index=False)

<h1> Ora real fuorisede ma partenze! </h1>

In [30]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique():
    n_pendolari = pendolari[pendolari['ResidenzaR']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['ResidenzaR']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
    n_tot = df[df['ResidenzaR']==prov]['Isc'].sum()
    if n_tot > 1000:
        r = real_fuorisede/n_tot*100
        provs.append(prov)
        rapps.append(r)

In [31]:
partenze_real = pd.DataFrame({
    'province':provs,
    'real_fuorisede/totale':rapps
}).sort_values(by='real_fuorisede/totale',ascending=False)

partenze_real['cod'] = partenze_real['province'].apply(lambda x: prov_cod[str(x)])

partenze_real.to_csv('province_partono_real_fuorisede.csv',index=False)

# Province con più fuorisede (non real)

In [105]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique(): #mettiamo come elenco unique sempre province di residenza 
                                        #perché più completo
    #n_pendolari = pendolari[pendolari['sede_ateneo']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['sede_ateneo']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede #-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
        
    n_tot = df[df['sede_ateneo']==prov]['Isc'].sum()
    #if n_tot > 1000:
    r = real_fuorisede/n_tot*100
    provs.append(prov)
    rapps.append(r)

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  


In [106]:
arrivi = pd.DataFrame({
    'province':provs,
    'fuorisede/totale':rapps
}).sort_values(by='fuorisede/totale',ascending=False).fillna(0)

arrivi['cod'] = arrivi['province'].apply(lambda x: prov_cod[x])
arrivi.to_csv('province_accolgono_più_fuorisede.csv',index=False)

# Province da cui partono più fuorisede (non real)

In [107]:
provs = []
rapps = []
for prov in df['ResidenzaR'].unique():
    #n_pendolari = pendolari[pendolari['ResidenzaR']==prov]['Isc'].sum()
    n_fuorisede = fuorisede[fuorisede['ResidenzaR']==prov]['Isc'].sum()
    real_fuorisede = n_fuorisede #-n_pendolari
    
    if real_fuorisede < 0:
        raise Exception('Qualcosa non va')
    n_tot = df[df['ResidenzaR']==prov]['Isc'].sum()
    if n_tot > 1000:
        r = real_fuorisede/n_tot*100
        provs.append(prov)
        rapps.append(r)

In [108]:
partenze = pd.DataFrame({
    'province':provs,
    'fuorisede/totale':rapps
}).sort_values(by='fuorisede/totale',ascending=False)

partenze['cod'] = partenze['province'].apply(lambda x: prov_cod[str(x)])

partenze.to_csv('province_partono_di_più.csv',index=False)

# merging arrivi

In [116]:
arrivi.merge(arrivi_real, on='cod').drop(columns='province_y').to_csv('arrivi_final.csv',index=False)

# merging partenze

In [117]:
partenze.merge(partenze_real, on='cod').drop(columns='province_y').to_csv('partenze_final.csv',index=False)

# temporary

In [38]:
unis_semidef = uni_fuorisede.merge(uni_fuorisede_real, on='Unis', how='outer')

In [49]:
unis_semidef['province limitrofe'] = unis_semidef['fuorisede/totale']-unis_semidef['real_fuorisede/totale']

In [51]:
unis_semidef.to_csv('uni_fuorisede_final.csv',index=False)